### ***Importando as bibliotecas***

In [33]:
import torch
import gc
import ipywidgets as widgets
from unsloth import FastLanguageModel
from transformers import TextStreamer
from IPython.display import display, clear_output

### ***Inicializando o Modelo***

**Prompt**

Para estruturar os dados de entrada do usuário vai ser utilizado o Alpaca, com isso vai ser possível fornecer ao modelo uma estrutura consistente que vai incluir:
- **Instrução**
- **Input (Contexto adicional)**

In [34]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

**Função de inicialização**

Recebe como parâmetro o input do usuário e o output do modelo:

**1.** Limpeza de memória.
- É realizado a limpeza na memória para o modelo que será carregado.

**2.** Inicialização do modelo.
- Inícia o modelo treinado.
- Habilita o modo de inferência

**3.** Criação do input e do streamer.
- Realiza a formatação do *alpaca_prompt* utilizando o parâmetro de entrada.

**4.** Gerar o texto
- Realiza a geração do novo texto.

In [50]:
# Inicia o modelo e realiza a geração de texto
def iniciar_modelo(input, output):
    with output:
        # Limpar o output anterior
        clear_output(wait=True)

        # Limpar a memória da GPU
        torch.cuda.empty_cache()
        gc.collect()
        
        max_seq_length = 256 # Choose any! We auto support RoPE Scaling internally!
        dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
        load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

        # Inicializa o modelo e o tokenizer
        model, tokenizer = FastLanguageModel.from_pretrained(
            model_name = './model/lora_model_final',
            max_seq_length = max_seq_length,
            dtype = dtype,
            load_in_4bit = load_in_4bit,
        )
        
        # Habilita o modo de inferência rápida
        FastLanguageModel.for_inference(model) # Enable native 2x faster inference
        
        # Instrução para o modelo
        instruction = 'Generate a detailed product description based on the following title:'
        
        # Cria o input para o modelo
        inputs = tokenizer(
            [
                alpaca_prompt.format(instruction, input, "")
            ], return_tensors = "pt").to("cuda")

        # Cria o streamer de texto
        text_streamer = TextStreamer(tokenizer)
        
        # Gera o texto
        _ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

### **Entrada Usuário**

O usuário pode digitar o nome do produto para que o modelo possa gerar uma descrição.

In [53]:
# Criando uma caixa de texto
texto = widgets.Text(
    description="PRODUCT NAME:",
    placeholder="TYPE HERE...",
    layout=widgets.Layout(width='30%'),
    style={'description_width': 'initial'}
)

botao = widgets.Button(
    description="SEARCH DESCRIPTION",
    button_style="primary",
    layout=widgets.Layout(width='30%')
)

output = widgets.Output(layout=widgets.Layout(width='100%'))

botao.on_click(lambda b: iniciar_modelo(texto.value, output))

# Exibindo a caixa de texto
display(texto, botao, output)

Text(value='', description='PRODUCT NAME:', layout=Layout(width='30%'), placeholder='TYPE HERE...', style=Text…

Button(button_style='primary', description='SEARCH DESCRIPTION', layout=Layout(width='30%'), style=ButtonStyle…

Output(layout=Layout(width='100%'))